In [1]:
'''
Project: Bangkit Capstone Project
Group: B21-CAP0194
Part: Modelling
'''

'\nProject: Bangkit Capstone Project\nGroup: B21-CAP0194\nPart: Modelling\n'

In [5]:
vocab_size = 10000
embedding_dim = 16
max_length = 32
trunc_type = 'post'
oov_tok = "<OOV>"

In [6]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
#Import Labelled Dataset
url = 'https://drive.google.com/file/d/1nOn8OONtZA7pcX4Pvh8W_wOUg49ZWTvT/view?usp=sharing'
file_id = url.split('/')[-2]
csv_url = 'https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(csv_url)
df.head()

,index,text,label
0,0,the ironies of the movement are too painfully...,private places
1,1,women in kuwait have launched a social media c...,public places
2,2,triger warning raising awareness of sexual ass...,private places
3,3,charges brought against the guy that smacked m...,private places
4,4,and the fourth presenter on 26th may is hannah...,educational places


In [4]:
df_cut = df[:100]

In [ ]:
df_cut.head(50)

,index,text
0,0,the ironies of the movement are too painfully...
1,1,women in kuwait have launched a social media c...
2,2,triger warning raising awareness of sexual ass...
3,3,charges brought against the guy that smacked m...
4,4,and the fourth presenter on 26th may is hannah...
5,5,this is so messed up are there any men in the...
6,6,to tackle an epidemic of male violence against...
7,7,there has been a new series of harassment alle...
8,8,want to see what the movement looks like its ...
9,9,female students dont want to have sex w dolan ...


In [ ]:
import getpass

APIKEY = getpass.getpass()

In [ ]:
# running Translate API
#from googleapiclient.discovery import build
#service = build('translate', 'v2', developerKey=APIKEY)

#df_cut['indo_text'] = df_cut['text'].apply(lambda x: (service.translations().list(source='en', target='id', q=x).execute())).apply(lambda x: x['translations']).apply(lambda x: x[0]).apply(lambda x: x['translatedText'])

In [ ]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

df['indo_text'] = df['text'].apply(lambda x: (service.translations().list(source='en', target='id', q=x).execute())).apply(lambda x: x['translations']).apply(lambda x: x[0]).apply(lambda x: x['translatedText'])

In [ ]:
df['indo_text'].head(50)

0     ironi gerakan ini terlalu menyakitkan dan jela...
1     perempuan di kuwait telah meluncurkan kampanye...
2     peringatan triger meningkatkan kesadaran akan ...
3     dakwaan yang diajukan terhadap pria yang memuk...
4     dan presenter keempat pada tanggal 26 mei adal...
5     Ini sangat kacau, adakah pria di mata publik, ...
6     untuk mengatasi epidemi kekerasan laki-laki te...
7     Ada serangkaian tuduhan pelecehan baru terhada...
8     ingin melihat seperti apa gerakan para wanita ...
9     siswa perempuan tidak ingin berhubungan seks d...
10    ketika kita berbicara tentang dan kekerasan, k...
11    ambil 221 juta tahun lalu di nih dana nih mand...
12    bantu siswa Anda untuk memerangi pelecehan men...
13    Saya masih ingin berbicara dengan wanita penya...
14    glenn greenwald tidak menyukai gerakan yang me...
15    untuk lebih lanjut tentang topik ini bergabung...
16    apa gerakan itu harus dilakukan dengan pembunu...
17    oped terbaru dari dan cuomo membutuhkan wa

In [ ]:
#Save DataFrame to CSV
df_translated = pd.DataFrame()
df_translated['index'] = df['index']
df_translated['text'] = df['text']
df_translated['indo_text'] = df['indo_text']
df_translated.to_csv("capstonetweettranslatednew.csv", index=False)

In [8]:
labels = pd.get_dummies(df.label)

In [9]:
df_new = pd.concat([df, labels], axis=1)
df_new = df_new.drop(columns='label')

In [10]:
tweet = df_new['text'].values
label = df_new[["private places","public places","work places","educational places", "cyber","neutral"]].values

In [11]:
label

array([[1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=uint8)

In [12]:
from sklearn.model_selection import train_test_split
#Train Text, Test Text, Train Label, Test Label
X_train, X_test, y_train, y_test = train_test_split(tweet, label, test_size=0.2, shuffle=True)

In [13]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X_train)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [14]:
import tensorflow as tf
'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.LSTM(64),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])

'''
model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])


transfer learning models for NLP: BERT, coba buka ini https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 16)            160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 128)           41472     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 244,966
Trainable params: 244,966
Non-trainable params: 0
_________________________________________________________________


In [16]:
num_epochs = 100
model.fit(padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_test))

Epoch 1/10
353/353 [==============================] - 51s 44ms/step - loss: 1.3053 - accuracy: 0.4396 - val_loss: 0.8691 - val_accuracy: 0.6644
Epoch 2/10
353/353 [==============================] - 14s 39ms/step - loss: 0.6898 - accuracy: 0.7535 - val_loss: 0.6569 - val_accuracy: 0.7845
Epoch 3/10
353/353 [==============================] - 14s 40ms/step - loss: 0.3709 - accuracy: 0.8721 - val_loss: 0.5743 - val_accuracy: 0.8221
Epoch 4/10
353/353 [==============================] - 14s 40ms/step - loss: 0.2323 - accuracy: 0.9195 - val_loss: 0.5847 - val_accuracy: 0.8296
Epoch 5/10
353/353 [==============================] - 14s 40ms/step - loss: 0.1630 - accuracy: 0.9474 - val_loss: 0.6740 - val_accuracy: 0.8122
Epoch 6/10
353/353 [==============================] - 14s 39ms/step - loss: 0.1102 - accuracy: 0.9657 - val_loss: 0.6479 - val_accuracy: 0.8444
Epoch 7/10
353/353 [==============================] - 14s 39ms/step - loss: 0.0740 - accuracy: 0.9814 - val_loss: 0.7112 - val_accuracy:

In [36]:
text = ["i meet my mom and my dad at home"]

tokenized_text = tokenizer.texts_to_sequences(text)

In [37]:
prediction = model.predict(tokenized_text)

In [38]:
labels = ["private places","public places","work places","educational places", "cyber","neutral"]
predicted = prediction.argmax(axis=1)
predict_labels = [labels[predict] for predict in predicted]
print(prediction)
print(predict_labels)

[[9.9998105e-01 7.0689789e-06 1.1788111e-05 7.7221586e-08 1.1246856e-09
  7.8202139e-10]]
['private places']


In [20]:
#Exporting the Model (TFLITE)
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

INFO:tensorflow:Assets written to: saved_model/1/assets


INFO:tensorflow:Assets written to: saved_model/1/assets


In [21]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [22]:
import pathlib

tflite_model_file = pathlib.Path('/content/model.tflite')
tflite_model_file.write_bytes(tflite_model)

1014760

In [23]:
from google.colab import files
files.download(tflite_model_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
#Exporting the Tokenized Word
import json

with open( 'word_dict.json' , 'w' ) as file:
    json.dump( tokenizer.word_index , file )